In [ ]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
#importing stuff

In [ ]:
def read_html(my_url):
    try:
        #reading the page html
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close
        #returns the page_html that's used for the html_reader function
        return page_html
    except Exception as e:
        #if the html is not valid, goes to this step
        print(e)
        print(my_url)
        return ''

In [ ]:
def html_reader(page_html):
    #returns the CEO Pay, median employee pay, and CEO to employee pay ratio of a company from an html
    try:
        page_soup = soup(page_html, 'html.parser')
        containers = page_soup.findAll("table",{"class":"table sa-table tablesaw-stack table-executive"})
        #finds all tables on the website (the CEO pay, median employee pay, and CEO to employee pay ratio are on a table)
        container = containers[2]
        #the container on the second index has the values we nee
        values = container.find_all("tbody")[0]
        #turn the table into readable text
        new_list = []
        for x in str(values.get_text()).split("\n"):
            if x != '':
                new_list.append(x)
        return new_list
        #run a for loop to remove all empty lists
        #This new list has CEO name on index 0, CEO pay on 2, Median employee pay on 4, and pay ratio on 6
    except Exception as e:
        #if the page_html is not valid, goes to this step
        print(e)
        print('html_reader error for' + my_url)    
        return ''

In [ ]:
#Testing if the html_reader and read_html functions work
html_reader(read_html('https://www1.salary.com/APPLE-INC-Executive-Salaries.html'))
#We want the values on index 2, 4, and 6

In [ ]:
#All of these names are already edited 
df = pd.read_excel (r'C:s&p500_companies_names_final.xlsx')
df

In [ ]:
#Fitting everything into the url on salary.com for constant scraping
#Anything not caught by these url changes will be manually replaced in the spreadsheet
'''
df['Company'] = df['Company'].str.replace('.com','-com')
df['Company'] = df['Company'].str.replace(' ','-')
df['Company'] = df['Company'].str.replace('.','')
df['Company'] = df['Company'].str.replace('-Class-A','')
df['Company'] = df['Company'].str.replace('-Class-B','')
df['Company'] = df['Company'].str.replace('-Class-C','')
df['Company'] = df['Company'].str.replace('-Class-P','')
df['Company'] = df['Company'].str.replace('oration','')
df['Company'] = df['Company'].str.replace('INCORPORATED','INC')
df['Company'] = df['Company'].str.replace('&','AND')
df['Company'] = df['Company'].str.replace(r"'",'')
df['Company'] = df['Company'].str.replace('COMPANY','CO')
df['Company'] = df['Company'].str.replace('!','')
df['Company'] = df['Company'].str.replace('INCORPORATED','INC')
df['Company'] = df['Company'].str.lower()
df
'''

In [ ]:
#Checking if the changes worked
for company in df['Company'].values:
    print(company)

In [ ]:
#Creating columns for the dataframe of -2 so we know if the value was not scraped
df = df.assign(CEO_Pay=-2)
df = df.assign(Median_Employee_Pay=-2)
df = df.assign(CEO_Pay_Ratio=-2)
df

In [ ]:
#Changes values in all three columns 
bad_link_list=[]
for index in range(len(df.index)):
    #Loops through all rows of the dataframe
    my_url='https://www1.salary.com/' + df.loc[index,'Company'] + '-Executive-Salaries.html'
    #Each URL follows the pattern above for scraping
    html_string=read_html(my_url)
    ceo_worker_pay_list=html_reader(html_string)
    if len(ceo_worker_pay_list)>6:
        df.at[index, 'CEO_Pay'] = ceo_worker_pay_list[2]
        #changes CEO_Pay column
        df.at[index, 'Median_Employee_Pay'] = ceo_worker_pay_list[4]
        #changes Median_Employee_Pay column
        df.at[index, 'CEO_Pay_Ratio'] = ceo_worker_pay_list[6]
        #changes CEO_Pay_Ratio column
    else:
        #makes a list of all links that did not work through the scrape
        bad_link_list.append(my_url)
    print(index)

In [ ]:
print(bad_link_list)

In [ ]:
len(bad_link_list)

In [ ]:
for element in bad_link_list:
    print(element)

In [ ]:
df

In [ ]:
df.to_csv('Example1.csv')